In [84]:
import numpy as np
import pandas as pd
from sklearn.metrics import dcg_score,ndcg_score
from collections import OrderedDict
import itertools
from scipy.spatial.distance import cosine
from sklearn.preprocessing import normalize


def cosine_sim(x, y):
    return 1 - cosine(x,y)

# **Welcome to week 4 project!**

Welcome to the last week of the course -- so excited to see that you've made it to the end! 👏 

We’ve already discussed the importance of measuring model performance. As Lord Kelvin said, “To measure is to know – If you cannot measure it, you cannot improve it.” And he was right – metrics are the only way we can actually evaluate our model’s performance!

In this week's project, we will touch upon two key aspects related to evaluation:
1. Behavioral metrics
2. Off-policy evaluation

For behavioral metrics, we will work with Spotify music sessions dataset, and implement a few behavioral metrics and understand their relationships with traditional metrics.

For off-policy evaluation, we will simulate a dataset where we have logged action policies, and see how IPS is implemented.


### Goals for this week's project:

For this week's project assignment, we will complete the following tasks:
1. Implement 3 behavioral metrics and present the correlation plot for them
2. Implement another ranking logic (e.g. sort by track popularity, or sort by danceability and compare all metrics on productional ranking logic and this new ranking logic.
3. Complete the implementation of two off-policy estimators: Capped IPS and Normalized Capped Importance Sampling (NCIS)


# Part A: Behavioral metrics

Behavioral metrics include factors like what items a user interacts with and how, the amount of time they spend on the platform, and how they spend that time.

To define and implement a few behavioral metrics, we will work with the Spotify music sessions dataset.
Download the dataset from GDrive: https://drive.google.com/drive/folders/10LGZMgXRuz2qPr_QDbYdVVlKEcnQ25YL?usp=sharing
(files: log_mini.cvs and tf_mini.csv)

## Spotify music sessions dataset

The public part of the dataset consists of roughly 130 million listening sessions with associated user interactions on the Spotify service. In total, users interacted with almost 4 million tracks during these sessions, and the dataset includes acoustic features and metadata for all of these tracks.

Detailed description of the dataset can be found here:
https://drive.google.com/file/d/1BELTuH4nBeyHna5EAGzJv-HWHKrbxPsf/view?usp=sharing



In [2]:
log = pd.read_csv("data/log_mini.csv")
tracks = pd.read_csv("data/tf_mini.csv")

In [3]:
log.columns

Index(['session_id', 'session_position', 'session_length', 'track_id_clean',
       'skip_1', 'skip_2', 'skip_3', 'not_skipped', 'context_switch',
       'no_pause_before_play', 'short_pause_before_play',
       'long_pause_before_play', 'hist_user_behavior_n_seekfwd',
       'hist_user_behavior_n_seekback', 'hist_user_behavior_is_shuffle',
       'hour_of_day', 'date', 'premium', 'context_type',
       'hist_user_behavior_reason_start', 'hist_user_behavior_reason_end'],
      dtype='object')

In [4]:
log.head(2)

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,1,20,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,False,False,False,True,0,0,...,0,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,2,20,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,False,False,False,True,0,1,...,0,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone


In [5]:
tracks.columns

Index(['track_id', 'duration', 'release_year', 'us_popularity_estimate',
       'acousticness', 'beat_strength', 'bounciness', 'danceability',
       'dyn_range_mean', 'energy', 'flatness', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mechanism', 'mode', 'organism', 'speechiness',
       'tempo', 'time_signature', 'valence', 'acoustic_vector_0',
       'acoustic_vector_1', 'acoustic_vector_2', 'acoustic_vector_3',
       'acoustic_vector_4', 'acoustic_vector_5', 'acoustic_vector_6',
       'acoustic_vector_7'],
      dtype='object')

In [6]:
tracks.head(2)

,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,...,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7
0,t_a540e552-16d4-42f8-a185-232bd650ea7d,109.706673,1950,99.975414,0.458040,0.519497,0.504949,0.399767,7.511880,0.817709,...,4,0.935512,-0.033284,-0.411896,-0.02858,0.349438,0.832467,-0.213871,-0.299464,-0.675907
1,t_67965da0-132b-4b1e-8a69-0ef99b32287c,187.693329,1950,99.969430,0.916272,0.419223,0.545530,0.491235,9.098376,0.154258,...,3,0.359675,0.145703,-0.850372,0.12386,0.746904,0.371803,-0.420558,-0.213120,-0.525795


### Traditional metrics: ndcg@k

We will use the log dataframe as the main dataframe for evaluation of metrics. The skip_1 flag can be used as a relevance signal -- if the user found the recommendation relevant, skip_1 = False. With this relevance signal, we can compute simple ndcg metrics -- one for each session and then averaged across all sessions. This will serve as a base metric for comparison.

Note: the ranking logic here is assumed to be the production ranker, i.e. sorting by session_position gives the exact order of tracks the Spotify ranker presented to the user.

Lets compute simple skip rate and ndcg metric for the production ranker:

In [10]:
topk = 10

skip_rate_session = log.groupby('session_id').apply(lambda x: x.nsmallest(topk,['session_position'])).reset_index(drop = True).groupby("session_id").skip_1.mean().mean()
print("average skip rate in the session: ",skip_rate_session)

average skip rate in the session:  0.40894


In [10]:
topk = 10

def get_ndcg(df):
    true_relevance = np.asarray(1-np.asarray(df['skip_1']*1.0))
    ranker_scores = np.asarray(1/np.asarray(df['session_position'])) # approximate the ranker scores using the session position
    return(ndcg_score([true_relevance], [ranker_scores]))


ndcg_metric = log.groupby('session_id').apply(lambda x: x.nsmallest(topk,['session_position'])).reset_index(drop = True).groupby("session_id").apply(get_ndcg).mean()
print("NDCG@k, with k= ",topk," is: ",ndcg_metric)

NDCG@k, with k=  10  is:  0.8330266041453138


## Goals for this week:

Implement a few behavioral metrics and compare their correlations. We will implement the following three metrics:
1. *Time to first skip:* how long did it take for the user to get the first bad recommendation, i.e. a recommendation they skipped. Since we can't easily calculate time, we can use number of songs as a proxy and compute the metric as number of songs it took for the first skip.

2. *Sustained dissatisfaction:* we assume that the user is dissatisfied in a sustained manner if they skip 3 songs consecutively.

3. *Session coherence:* we define coherence as how similar the recommended musical tracks are. We can use the acoustic_vector of the music tracks to calculate the similarity.

In [112]:
# Making this because this seems to be recurrent throughout all the computed 
# metrics
def compute_topk(log_df, topk, fn):
    return log.groupby('session_id') \
    .apply(lambda x: x.nsmallest(topk,['session_position'])) \
    .reset_index(drop = True) \
    .groupby("session_id") \
    .apply(fn)

In [113]:
def get_ndcg(log_df, topk):
    def ndcg(session_df):
        true_relevance = np.asarray(1-np.asarray(session_df['skip_1']*1.0))
        ranker_scores = np.asarray(1/np.asarray(session_df['session_position'])) # approximate the ranker scores using the session position
        return ndcg_score([true_relevance], [ranker_scores])
    
    return compute_topk(log_df, topk, ndcg)

print(f"Avg NDCG @{5}: ", get_ndcg(log, 5).mean())
print(f"Avg NDCG @{10}: ", get_ndcg(log, 10).mean())


Avg NDCG @5:  0.8235700606177565
Avg NDCG @10:  0.8330266041453138


In [120]:
# implement session metric 1: time to first skip (number of songs to first skip)

def get_time_to_first_skip(log_df, topk):
    def time_to_first_skip(session_df):
        pos = session_df.loc[session_df["skip_1"] == True, "session_position"].min()
        return topk+1 if np.isnan(pos) else pos
    
    return compute_topk(log_df, topk, time_to_first_skip)
            
print(f"Avg time to first skip @{5}: ", get_time_to_first_skip(log, 5).mean())
print(f"Avg time to first skip @{10}: ", get_time_to_first_skip(log, 10).mean())


Avg time to first skip @5:  3.3585
Avg time to first skip @10:  4.2057


In [115]:
# implement session metric 2: sustained dissatisfaction: proportions of sessions with 3 consecutive skips
def get_sustained_dissatisfaction(log_df, topk, num_consecutive_sess=3):
    def sustained_dissatisfaction(session_df):
        return (session_df.sort_values(["session_position"], ascending=True) \
                .rolling(num_consecutive_sess)["skip_1"] \
                .sum() == num_consecutive_sess).sum()
    return compute_topk(log_df, topk, sustained_dissatisfaction)

print(f"Avg Sustained Dissatisfaction @{5}:", get_sustained_dissatisfaction(log, 5).mean())
print(f"Avg Sustained Dissatisfaction @{topk}:", get_sustained_dissatisfaction(log, topk).mean())


Avg Sustained Dissatisfaction @5: 0.5641
Avg Sustained Dissatisfaction @5: 0.5641


In [85]:
# implement session metric 3: session coherence: average similarity between the top recommended tracks
# stiching all acounstic_vector_[0-7]

acoustic_vectors_cols = [col for col in tracks if col.startswith("acoustic_vector")]
acoustic_vectors = tracks[acoustic_vectors_cols].values
# Need to normalize these vectors over axis 1, otherwise the cosine similarity with be > 1.
normalized_acoustic_vectors = normalize(acoustic_vectors, axis=1, norm='l1')
ACOUSTIC_VECTORS_DICT = dict(zip(tracks["track_id"], normalized_acoustic_vectors))

In [116]:
def get_session_coherence(log_df, topk):
    def session_coherence(session_df):
        track_pairs = itertools.combinations(session_df["track_id_clean"], 2)
        track_pairs_sim = [cosine_sim(ACOUSTIC_VECTORS_DICT[t1], ACOUSTIC_VECTORS_DICT[t2]) for (t1, t2) in track_pairs]
        return np.mean(track_pairs_sim)
    
    return compute_topk(log_df, topk, session_coherence)

print(f"Avg Session Coherence @{5}:", get_session_coherence(log, 5).mean())
print(f"Avg Session Coherence @{10}:", get_session_coherence(log, 10).mean())

Avg Session Coherence @5: 0.8234136236006494
Avg Session Coherence @10: 0.8174122151803728


Once these metrics are implemented, compute these metrics for topk=5 and topk=10 compare their estimates for the production ranker as a correlation plot. Please note which metrics are correlated with ndcg metric.

**Additional goal:**
Implement another simple ranking logic, and compare the performance of both the production ranker and new ranking policy on the ndcg and three behavioral metrics.
A simple ranking policy could include sortby track popularity, or sort by danceability score for the track.

Please report the performance of these rankers on all four metrics.

In [117]:
topk = 5
corr_mat_df = pd.DataFrame({
    f"ndcg@{topk}": get_ndcg(log, topk),
    f"avg_first_skip_time@{topk}": get_time_to_first_skip(log, topk),
    f"avg_sustained_dissatisfaction@{topk}": get_sustained_dissatisfaction(log, topk),
    f"avg_session_coherence@{topk}": get_session_coherence(log, topk)
})



In [118]:
corr_mat_df.corr()

,ndcg@5,avg_first_skip_time@5,avg_sustained_dissatisfaction@5,avg_session_coherence@5
ndcg@5,1.000000,0.652825,-0.742046,0.017673
avg_first_skip_time@5,0.652825,1.000000,-0.538216,0.017854
avg_sustained_dissatisfaction@5,-0.742046,-0.538216,1.000000,-0.011208
avg_session_coherence@5,0.017673,0.017854,-0.011208,1.000000


In [119]:
topk = 10
corr_mat_df = pd.DataFrame({
    f"ndcg@{topk}": get_ndcg(log, topk),
    f"avg_first_skip_time@{topk}": get_time_to_first_skip(log, topk),
    f"avg_sustained_dissatisfaction@{topk}": get_sustained_dissatisfaction(log, topk),
    f"avg_session_coherence@{topk}": get_session_coherence(log, topk)
})
corr_mat_df.corr()


,ndcg@10,avg_first_skip_time@10,avg_sustained_dissatisfaction@10,avg_session_coherence@10
ndcg@10,1.000000,0.592423,-0.591926,0.017214
avg_first_skip_time@10,0.592423,1.000000,-0.455600,0.010851
avg_sustained_dissatisfaction@10,-0.591926,-0.455600,1.000000,-0.003433
avg_session_coherence@10,0.017214,0.010851,-0.003433,1.000000


### Adding a simple reranker and relogging the metrics

In [121]:
## to rerank we will need to first merge the log and trancks data on train_id
reranked_log = pd.merge(log, tracks[["track_id", "us_popularity_estimate"]], left_on="track_id_clean", right_on="track_id")
reranked_log["session_position"] = reranked_log.groupby("session_id")["us_popularity_estimate"].rank("first", ascending=True)

In [122]:
print(f"Avg NDCG @{5}: ", get_ndcg(reranked_log, 5).mean())
print(f"Avg NDCG @{10}: ", get_ndcg(reranked_log, 10).mean())
            
print(f"Avg time to first skip @{5}: ", get_time_to_first_skip(reranked_log, 5).mean())
print(f"Avg time to first skip @{10}: ", get_time_to_first_skip(reranked_log, 10).mean())

print(f"Avg Sustained Dissatisfaction @{5}:", get_sustained_dissatisfaction(reranked_log, 5).mean())
print(f"Avg Sustained Dissatisfaction @{topk}:", get_sustained_dissatisfaction(reranked_log, topk).mean())


print(f"Avg Session Coherence @{5}:", get_session_coherence(reranked_log, 5).mean())
print(f"Avg Session Coherence @{10}:", get_session_coherence(reranked_log, 10).mean())


Avg NDCG @5:  0.8235700606177565
Avg NDCG @10:  0.8330266041453138
Avg time to first skip @5:  3.3585
Avg time to first skip @10:  4.2057
Avg Sustained Dissatisfaction @5: 0.5641
Avg Sustained Dissatisfaction @10: 1.7978
Avg Session Coherence @5: 0.8234136236006494
Avg Session Coherence @10: 0.8174122151803728


# Part B: Off Policy Evaluation

We log listener behavior based on the recommendations that the production recommender serves to the listener. Using this data to assess any new recommender system, however, can present challenges – the production recommender and the new recommender can drastically differ in the results that they display to the user. For example, maybe the new recommender presents a lot of niche content, while the production recommender presents a lot of popular options. This can be an issue when evaluating a new recommender – If you don’t have any feedback on a recommendation because you never presented it to a user, how can you evaluate whether it’s a good recommendation?
If you have a new policy to test that’s very similar to your old approach, then this won’t be an issue, and it’ll be easy to test! However, if the policy is very different, then you’ll need to collect special logged data.

In this part of the project, we will simulate a recommendation policy and leverage counterfactual estimators as metrics to compare performance.


Lets first begin by generating a few users and products. For ease of simulation, we assume users derive equal satisfaction from each item.

In [123]:

users = np.array(["user1", "user2", "user3"])
products = np.array(
    [
        "product_a",
        "product_b",
        "product_c",
        "product_d",
        "product_e",
        "product_f",
        "product_g",
    ]
)

satisfaction = {
    "product_a": 100,
    "product_b": 150,
    "product_c": 100,
    "product_d": 200,
    "product_e": 500,
    "product_f": 120,
    "product_g": 160,
}



Lets also implement whether a given user will accept a given recommendation or not. Once done, we can implement a target policy that makes recommendations.

In [124]:

def will_purchase(user, product):
    if user == "user1" and (
        product == "product_a" or product == "product_b" or product == "product_c"
    ):
        return True
    elif user == "user2" and (product == "product_d" or product == "product_e"):
        return True
    elif user == "user3" and (product == "product_f" or product == "product_g"):
        return True
    else:
        return False


def choose_user():
    return np.random.choice(users, size=1)


def logging_policy():
    return np.random.choice(products, size=1), 1 / len(products)


class TargetPolicy:
    def __init__(self):
        self.user_probs = {
            "user1": np.array([0.1, 0.1, 0.2, 0.1, 0.15, 0.15, 0.20]),
            "user2": np.array([0.1, 0.10, 0.05, 0.25, 0.3, 0.1, 0.1]),
            "user3": np.array([0.06, 0.06, 0.3, 0.06, 0.06, 0.4, 0.06]),
        }

        for user, probs in self.user_probs.items():
            assert probs.sum() == 1
            assert len(probs) == len(products)

    def recommend(self, user):
        user_prob = self.user_probs[user]
        product = np.random.choice(products, size=1, p=user_prob)
        product_idx = np.where(products == product)
        prob = user_prob[product_idx]

        return product, prob

    def get_prob(self, user, product):
        user_prob = self.user_probs[user]
        product_idx = np.where(products == product)
        product_prob = user_prob[product_idx]

        return product_prob



Having defined all key components of the dataset generation, lets create logged data that we can finally use for evaluation purposes:

In [125]:
def compute_satisfaction(user, product):
    if will_purchase(user, product):
        return satisfaction[product.item()]
    else:
        return 0


def create_logs(n=1000):
    logs = []
    target_policy = TargetPolicy()

    for _ in range(n):
        user = choose_user()

        logging_product, logging_prob = logging_policy()
        model_prob = target_policy.get_prob(user.item(), logging_product)

        target_product, _ = target_policy.recommend(user.item())

        logging_satisfaction = compute_satisfaction(user, logging_product)
        target_satisfaction = compute_satisfaction(user, target_product)

        log = OrderedDict(
            {
                "user_features": user.item(),
                "item_placed": logging_product.item(),
                "item_prob": logging_prob,
                "item_satisfaction": logging_satisfaction,
                "model_prob": model_prob.item(),
                "ab_test_satisfaction": target_satisfaction,
            }
        )

        logs.append(log)

    return pd.DataFrame(logs)

Here is what ur logged data now looks like:

In [126]:
logs = create_logs(n=1000)
logs.head(5)

,user_features,item_placed,item_prob,item_satisfaction,model_prob,ab_test_satisfaction
0,user3,product_f,0.142857,120,0.4,120
1,user2,product_b,0.142857,0,0.1,0
2,user1,product_b,0.142857,150,0.1,0
3,user1,product_a,0.142857,100,0.1,0
4,user2,product_b,0.142857,0,0.1,0


In [127]:
logs[["user_features", "item_placed", "item_prob", "item_satisfaction"]].sample(n=10)

,user_features,item_placed,item_prob,item_satisfaction
173,user2,product_g,0.142857,0
247,user1,product_e,0.142857,0
9,user2,product_d,0.142857,200
486,user1,product_g,0.142857,0
328,user1,product_b,0.142857,150
813,user1,product_f,0.142857,0
60,user1,product_e,0.142857,0
300,user1,product_e,0.142857,0
783,user2,product_a,0.142857,0
599,user1,product_e,0.142857,0


With all the dataset ready, lets compute the mean rewards (satisfaction) for the logging/production policy and the target policy:

In [128]:
%%time
sim = create_logs(n=100000)
logging_policy = sim["item_satisfaction"].mean()
target_policy = sim["ab_test_satisfaction"].mean()

print(f"Expected reward from logging policy: {logging_policy: .2f}")
print(f"Expected reward from target policy: {target_policy: .2f}")

Expected reward from logging policy:  63.32
Expected reward from target policy:  101.54
CPU times: user 3.15 s, sys: 65.1 ms, total: 3.22 s
Wall time: 3.2 s


Now lets implement the IPS estimator:

In [129]:
def compute_ips(df):
    assert {"model_prob", "item_prob", "item_satisfaction"}.issubset(df.columns)
    return (df["model_prob"] / df["item_prob"] * df["item_satisfaction"]).mean()

In [130]:
ips_est = compute_ips(logs)
ips_est

95.641

Computing the IPS estimator on our 1,000 entry log gives an average revenue of 109.34 (very close to the true performance of 100.99) compared with the average revenue of the logging policy of 63.36. Therefore, we should be confident to deploy our target policy to production and do an AB test comparing it with the logging policy as a final validation.

## Goals for this part of project:

Finish the implementation of two additional off-policy estimators:
1. Capped IPS
2. Normalized Capped Importance Sampling (NCIS)

Feel free to try different capping thresholds, and compare the reward and standard deviations of these estimators with the IPS estimator and mean reward.


In [133]:
def compute_capped_ips(df, cap=1000):
    assert {"model_prob", "item_prob", "item_satisfaction"}.issubset(df.columns)
    return (np.clip(df["model_prob"]/df["item_prob"], a_min=0, a_max=cap)* df["item_satisfaction"]).mean()

cips_est = compute_capped_ips(logs)
cips_est

95.641

In [136]:
def compute_ncis(df):
    # Formula: https://speakerdeck.com/alpicola/b-testing-for-recommender-systems?slide=17
    nume = (df["model_prob"] / df["item_prob"] * df["item_satisfaction"]).sum()
    deno =  (df["model_prob"] / df["item_prob"]).sum()
    return nume/deno

ncis_est = compute_ncis(logs)
ncis_est

95.06018228623113